# AWS SageMakerで実行するためのnotebook
* [SageMakerとGitHubの連携方法](https://predictivehacks.com/aws-sagemaker-and-github-integration/)

In [3]:
%%capture
!pip install wandb

In [39]:
import sagemaker
from sagemaker.estimator import Estimator

session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [40]:
import wandb

wandb.login()
settings = wandb.setup().settings

current_api_key = '9fc1a204d5b89f2818dab1b9d83677db2be23972'
wandb.wandb_lib.apikey.api_key(settings = settings)

'9fc1a204d5b89f2818dab1b9d83677db2be23972'

In [41]:
pwd

'/home/ec2-user/SageMaker/kaggle-Feedback-English-Language-Learning'

In [42]:
import boto3
s3 = boto3.resource("s3") 
bucket = s3.Bucket("fuyu-bucket")

bucket.upload_file('/home/ec2-user/SageMaker/kaggle-Feedback-English-Language-Learning/src/config.yaml',
                   'feedback-prize-english-language-learning/yaml/config.yaml')

In [52]:
estimator = Estimator(
    # AWSの用意したdocker image
    # conda環境なのでpipを使うと壊れてしまう可能性がある
    #image_uri="763104351884.dkr.ecr.ap-northeast-1.amazonaws.com/pytorch-training:1.12.0-cpu-py38-ubuntu20.04-sagemaker",
    # 自作のdocker image
    image_uri='692139951728.dkr.ecr.ap-northeast-1.amazonaws.com/fuyu:pytroch',
    role=role,
    #instance_type="ml.m5.xlarge",
    instance_type = 'ml.p3.2xlarge',
    instance_count=1,
    code_location="s3://fuyu-bucket/sagemaker/logs", 
    output_path="s3://fuyu-bucket/sagemaker/logs", 
    sagemaker_session=session,
    #hyperparameters = 'src/config.yaml',
    entry_point="src/train-argparse.py",
    environment = {'WANDB_API_KEY':'9fc1a204d5b89f2818dab1b9d83677db2be23972',
                  "GPU_NUM_DEVICES": "1"}
)


estimator.fit({
    # 入力データの設定
    # /opt/ml/input/data/train_dataにコピーされる
    "train_data": "s3://fuyu-bucket/feedback-prize-english-language-learning/cohesion.csv",
   # "yaml" : 's3://fuyu-bucket/feedback-prize-english-language-learning/yaml/config.yaml'
})

2022-10-02 18:42:09 Starting - Starting the training job...ProfilerReport-1664736128: InProgress
...
2022-10-02 18:42:52 Starting - Preparing the instances for training.........
2022-10-02 18:44:32 Downloading - Downloading input data...
2022-10-02 18:45:01 Training - Training image download completed. Training in progress..2022-10-03 03:45:04,253 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train_data": "/opt/ml/input/data/train_data"
    },
    "current_host": "algo-1",
    "current_instance_group": "homogeneousCluster",
    "current_instance_group_hosts": [
        "algo-1"
    ],
    "current_instance_type": "ml.p3.2xlarge",
    "distribution_hosts": [],
    "distribution_instance_groups": [],
    "framework_module": null,
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {},
    "input_config_dir": "/opt/ml/input/config",
    "input_data_config": {
        "train

KeyboardInterrupt: 